In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



In [ ]:
# test import
print("TensorFlow v" + tf.__version__)
print("TensorFlow Decision Forests v" + tfdf.__version__)

## Load the Dataset

In [ ]:
# load dataset into a Pandas DataFrame
dataset_df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
print("Full train dataset shape is {}".format(dataset_df.shape))

In [ ]:
# display the first 5 examples
dataset_df.head(5)

## Basic Exploration of the Dataset

In [ ]:
dataset_df.describe()

In [ ]:
dataset_df.info()

## Bar Chart for label column:
### Transported

In [ ]:
# Transported ?
plot_df = dataset_df.Transported.value_counts()
plot_df.plot(kind = "bar")

## Numerical data distribution

In [ ]:
fig, ax = plt.subplots(5, 1, figsize = (10, 10))
plt.subplots_adjust(top = 2)

sns.histplot(dataset_df['Age'], color = 'blue', bins = 50, ax = ax[0] );
sns.histplot(dataset_df['FoodCourt'], color = 'blue', bins = 50, ax = ax[1]);
sns.histplot(dataset_df['ShoppingMall'], color = 'blue', bins = 50, ax = ax[2]);
sns.histplot(dataset_df['Spa'], color = 'blue', bins = 50, ax = ax[3]);
sns.histplot(dataset_df['VRDeck'], color = 'blue', bins = 50, ax = ax[4]);




## Prepare the Dataset
drop both PassengerId and Name columns as they are not necessary for model training

In [ ]:
dataset_df = dataset_df.drop(['PassengerId', 'Name'], axis = 1)
dataset_df.head(5)

## Check for the missing values:

In [ ]:
dataset_df.isnull().sum().sort_values(ascending = False)

This dataset contains a mix of numeric, categorical and missing features.

TF-DF supports all these feature types natively, and no preprocessing is required.

But this dataset also has boolen fields with missing values.
TF-DF doesn't support boolean fields yet.
So we need to convert those field into int
To account for the missing values in the boolean fields, we will replace them with zero.

In this notebook, we will replace null value entries with zero for numerical columns as well and only let TF-Df handle the missing values in caregorical columns.

Note: You can choose to let TF-DF handle missing values in numerical columns if need be.

In [ ]:
dataset_df[['VIP', 'CryoSleep', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = dataset_df[['VIP', 'CryoSleep', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(value = 0)
dataset_df.isnull().sum().sort_values(ascending = False)

Since, TF-Df cannot handle boolean columns, we will hae to dajust the labels in column 'Transported', 'CryoSleep' and 'VIP' to conver them into the integer format that TF-DF expects.



In [ ]:
label = "Transported"
dataset_df[label] = dataset_df[label].astype(int)

dataset_df['VIP'] = dataset_df['VIP'].astype(int)
dataset_df['CryoSleep'] = dataset_df['CryoSleep'].astype(int)


The value of column 'Cabin' is string with the fomat 'Deck/Cabin_num/Side'.
Here we will split the Cabin column and create 3 new columns 'Deck', 'Cabin_num' and 'Side', since it will be easier to train the model on those individual data.

Run the following command to split the column Cabin into columns Deck,Cabin_num and Side

In [ ]:
dataset_df[["Deck", "Cabin_num", "Side"]] = dataset_df["Cabin"].str.split("/", expand = True)

Remove original Cabin column from the dataset since it is not needed anymore

In [ ]:
try:
    dataset_df = dataset_df.drop('Cabin', axis = 1)
except KeyError:
    print("Field does not exist")

Display first 5 examples from the prepared dataset

In [ ]:
dataset_df.head(5)

## Split the dataset into training and test datasets

In [ ]:
def split_dataset(dataset, test_ratio = 0.20):
    test_indices = np.random.rand(len(dataset)) < test_ratio
    return dataset[~test_indices], dataset[test_indices]

train_ds_pd, valid_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples in testing.".format(len(train_ds_pd), len(valid_ds_pd)))


    

There is no more step required before we can train the model.
We need convert the dataset from Pandas format into TensorFlow Datasets format.


In [ ]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label = label)
valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_ds_pd, label = label)

## Select a Model

To start, we will work with a Random Forest.
We can list all the available models in TensorFlow Decision Forests using the following code:


In [ ]:
tfdf.keras.get_all_models()

## Configure the Model
We will use the defaults to create the Random Forest Model.
By default the model is set to train for a classification task


In [ ]:
rf = tfdf.keras.RandomForestModel()
# optional, you can use this to include a list of eval merics
rf.compile(metrics = ["accuracy"])

## Train the model
we will train the model using a one-liner
the waring about Autograoh can be ignored

In [ ]:
rf.fit(x = train_ds)

## Visualize the model
one benefit to tree-based models is that we can easily visualize them.
The default number of trees used in the Random Forests is 300.
We can select a tree to display below.

In [ ]:
tfdf.model_plotter.plot_model_in_colab(rf, tree_idx = 0, max_depth = 3)

## 

## Evaluate the Model on the Out of Bag(OOB) data and te validation dataset

Before training the dataset we have manually seperated 20% of the dataset for validation nameds as valid_ds.

We can also use Out of Bag(OOB) score to validate our RandomForestModel.
To train a Random Forest Model, a set of random samples from training set are choosen by the alogorihm and the rest of the samples are used to finetune the model.
The subset of data that is not chosen is known as Out of Bag data.
OOB score is computed on the OOb data.

The training logs show the accurary evaluated on the out of bag dataset according to the number of trees in the model.

**Note: Larger values are better fot this hyperparameter**

In [ ]:
import matplotlib.pyplot as plt
logs = rf.make_inspector().training_logs()
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of Trees")
plt.ylabel("Accuracy (Out-Of-Bag)")
plt.show()

In [ ]:
inspector = rf.make_inspector()
inspector.evaluation()

now, run an evaluation using the validation dataset

In [ ]:
evaluation = rf.evaluate(x = valid_ds, return_dict = True)

for name, value in evaluation.items():
    print(f"{name}: {value:.4f}")

## Variable importances
Variable importances generally indicate how much a feature contributes to the model predictions or quality.
There are several ways to identify important features using TensorFlow Decision Forests.
Firstly, list the available Variable Importances for Decision Tress:


In [ ]:
print(f"Available Variable Importances:")

for importance in inspector.variable_importances().keys():
    print("\t", importance)

As an example, display the important features for the Variable Importance NUM_AS_ROOT.

The larger the importance score for NUM_AS_ROOT, the more impact it has on the out come of the model.

By default, the list is sorted from the most important to the least.
From the output you can infer that the feature at the top of the list is used as the root node in most number of trees in the random forest than any other feature.


In [ ]:
# each line is : (feature name, (index of the feature), importance score)
inspector.variable_importances()["NUM_AS_ROOT"]

## Submission

In [ ]:
# Load the test dataset
test_df = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
submission_id = test_df.PassengerId

# Replace NaN values with zero
test_df[['VIP', 'CryoSleep']] = test_df[['VIP', 'CryoSleep']].fillna(value=0)

# Creating New Features - Deck, Cabin_num and Side from the column Cabin and remove Cabin
test_df[["Deck", "Cabin_num", "Side"]] = test_df["Cabin"].str.split("/", expand=True)
test_df = test_df.drop('Cabin', axis=1)

# Convert boolean to 1's and 0's
test_df['VIP'] = test_df['VIP'].astype(int)
test_df['CryoSleep'] = test_df['CryoSleep'].astype(int)

# Convert pd dataframe to tf dataset
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df)

# Get the predictions for testdata
predictions = rf.predict(test_ds)
n_predictions = (predictions > 0.5).astype(bool)
output = pd.DataFrame({'PassengerId': submission_id,
                       'Transported': n_predictions.squeeze()})

output.head()

In [ ]:
sample_submission_df = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')
sample_submission_df['Transported'] = n_predictions
sample_submission_df.to_csv('/kaggle/working/submission.csv', index = False)
sample_submission_df.head()